In [1]:
%matplotlib inline

In [2]:
from ipywidgets import FloatProgress
from IPython.display import display

In [3]:
import pandas as pd
import fuzzywuzzy.process as fwp

In [4]:
mfg_firm = pd.read_csv('../data/MfgFirm.csv')

In [5]:
mfg_firm.size

17976

In [9]:
mfg_firm.head()

,RecallID,RecallNumber,MfgFirm,caseID
0,8223,18073,"Western Gas Partners LP, of The Woodlands, Texas",1
1,8221,18711,"HealthPostures LLC, Prior Lake, Minn.",2
2,8220,18069,"Rocky Mountain Bicycles, of Canada",3
3,8218,18710,"Golden Chang, of China",4
4,8219,18071,"Golden Forest, of Vietnam",5


In [10]:
us_company_codes = pd.read_excel('../data/All US Company codes in COMPUSTAT.xlsx')

In [11]:
us_company_codes.size

1437360

In [10]:
us_company_codes.head()

,Global Company Key,Data Date,Data Year - Fiscal,Industry Format,Level of Consolidation - Company Annual Descriptor,Population Source,Data Format,Ticker Symbol,CUSIP,Company Name,ISO Currency Code,CIK Number,Active/Inactive Status Marker,Current ISO Country Code - Incorporation,County Code
0,1004,05/31/2010,2009.0,INDL,C,D,STD,AIR,361105,AAR CORP,USD,1750.0,A,USA,NaN
1,1004,05/31/2011,2010.0,INDL,C,D,STD,AIR,361105,AAR CORP,USD,1750.0,A,USA,NaN
2,1004,05/31/2012,2011.0,INDL,C,D,STD,AIR,361105,AAR CORP,USD,1750.0,A,USA,NaN
3,1004,05/31/2013,2012.0,INDL,C,D,STD,AIR,361105,AAR CORP,USD,1750.0,A,USA,NaN
4,1004,05/31/2014,2013.0,INDL,C,D,STD,AIR,361105,AAR CORP,USD,1750.0,A,USA,NaN


In [11]:
mfg_firm_names = mfg_firm[['MfgFirm']]
mfg_firm_names.head()

,MfgFirm
0,"Western Gas Partners LP, of The Woodlands, Texas"
1,"HealthPostures LLC, Prior Lake, Minn."
2,"Rocky Mountain Bicycles, of Canada"
3,"Golden Chang, of China"
4,"Golden Forest, of Vietnam"


In [12]:
company_codes__no_duplicates = us_company_codes[['Company Name']].drop_duplicates(keep='last')
company_codes__no_duplicates.head()

,Company Name
7,AAR CORP
8,ADC TELECOMMUNICATIONS INC
16,AMERICAN AIRLINES GROUP INC
23,CECO ENVIRONMENTAL CORP
39,ASA GOLD AND PRECIOUS METALS


In [13]:
company_names = list(set(company_codes__no_duplicates['Company Name']))
len(company_names)

15989

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [15]:
fuzz.token_sort_ratio('Western Gas Partners LP, of The Woodlands, Texas', 'WESTERN GAS PARTNERS LP')

67

In [17]:
name = 'Western Gas Partners LP, of The Woodlands, Texas'
process.extractOne(name, company_names)

('WESTERN GAS PARTNERS LP', 90)

In [18]:
name = 'HealthPostures LLC, Prior Lake, Minn.'
process.extractBests(name, company_names)

[('LIN MEDIA LLC', 86),
 ('GYRODYNE LLC', 86),
 ('CITYCENTER HOLDINGS LLC', 86),
 ('JEFFERIES GROUP LLC', 86),
 ('CHRYSLER GROUP LLC', 86)]

In [40]:
f = FloatProgress(min=0, max=mfg_firm_names.size)
display(f)

FUZZYWUZZY_THREASHOLD = 90


mapping = dict()
for row in mfg_firm_names.iterrows():
    mfg_firm_name = row[1]['MfgFirm']
    
    # fuzzywuzzy match with company_names
    company_name, score = process.extractOne(mfg_firm_name, company_names)
    mapping[mfg_firm_name] = (company_name, score) if score >= FUZZYWUZZY_THREASHOLD else (None, score)
    
    f.value += 1
     

FloatProgress(value=0.0, max=4494.0)

In [41]:
mapping

{'Western Gas Partners LP, of The Woodlands, Texas': ('WESTERN GAS PARTNERS LP',
  90),
 'HealthPostures LLC, Prior Lake, Minn.': (None, 86),
 'Rocky Mountain Bicycles, of Canada': (None, 86),
 'Golden Chang, of China': (None, 86),
 'Golden Forest, of Vietnam': (None, 86),
 'American Honda Motor Company Inc., of Torrance, Calif.': (None, 86),
 'Deere & Company, of Moline, Ill.': ('DEERE & CO', 90),
 'Yueqing City Island Fiber Optic Christmas Tree Plant, of China': (None, 86),
 'Polaris Industries Inc., of Medina, Minn.': ('POLARIS INDUSTRIES INC', 90),
 'Manu Changzhou Future International Business Mafacturer, from China': (None,
  86)}

In [37]:
def set_company_name(row):
    mfg_firm_name = row.MfgFirm
    return mapping[mfg_firm_name]
        
mfg_firm['company_name'] = mfg_firm.apply(set_company_name, axis=1)

KeyError: ('ABEST Technology Co., Ltd., of China', 'occurred at index 10')

In [35]:
mfg_firm

,RecallID,RecallNumber,MfgFirm,caseID,company_name
0,8223,18073,"Western Gas Partners LP, of The Woodlands, Texas",1,WESTERN GAS PARTNERS LP
1,8221,18711,"HealthPostures LLC, Prior Lake, Minn.",2,LIN MEDIA LLC
2,8220,18069,"Rocky Mountain Bicycles, of Canada",3,BANK OF MARIN BANCORP
3,8218,18710,"Golden Chang, of China",4,CHINA FUND INC
4,8219,18071,"Golden Forest, of Vietnam",5,BANK OF MONTREAL
5,8216,18072,"American Honda Motor Company Inc., of Torrance...",6,NORBORD INC
6,8215,18709,"Deere & Company, of Moline, Ill.",7,DEERE & CO
7,8212,18707,Yueqing City Island Fiber Optic Christmas Tree...,8,CHINA LODGING GROUP LTD -ADR
8,8213,18708,"Polaris Industries Inc., of Medina, Minn.",9,POLARIS INDUSTRIES INC
9,8211,18066,Manu Changzhou Future International Business M...,10,VANGUARD TOTAL INTERNATIONAL


In [62]:
merged = pd.merge(mfg_firm, 
                  us_company_codes,
                  left_on='company_name',
                  right_on='Company Name',
                  suffixes=['_firm_names','_company_codes'],
                  how='left')

,MfgFirm,firm_code,Company Name
0,"Western Gas Partners LP, of The Woodlands, Texas",WESTERN GAS PARTNERS LP,WESTERN GAS PARTNERS LP
1,"HealthPostures LLC, Prior Lake, Minn.",ATLAS ENERGY GROUP LLC,ATLAS ENERGY GROUP LLC
2,"Rocky Mountain Bicycles, of Canada",GENWORTH MI CANADA INC,GENWORTH MI CANADA INC
3,"Golden Chang, of China",CHINA FUND INC,CHINA FUND INC
4,"Golden Forest, of Vietnam",SINO-FOREST CORP,SINO-FOREST CORP
5,"American Honda Motor Company Inc., of Torrance...",BLUCORA INC,BLUCORA INC
6,"Deere & Company, of Moline, Ill.",DEERE & CO,DEERE & CO
7,Yueqing City Island Fiber Optic Christmas Tree...,U S CHINA MINING GROUP INC,U S CHINA MINING GROUP INC
8,"Polaris Industries Inc., of Medina, Minn.",POLARIS INDUSTRIES INC,POLARIS INDUSTRIES INC
9,Manu Changzhou Future International Business M...,MAGNA INTERNATIONAL INC,MAGNA INTERNATIONAL INC
